# 0.简介

**一枚热爱技术的菜鸡，今天想做一个目标检测和嵌入式的结合体，做一个jetson nano乐器**

![](https://ai-studio-static-online.cdn.bcebos.com/a85fbc34eda64015a2bada1221cc9a1be300784ca9204613b3e9b94511f5b0a4)

**方案：**

**使用paddledetection训练模型并使用paddleInference在jetson nano上部署目标检测网络，**

**使用Tensorrt进行Jetsonnano的加速，**

**来挽救我的2GB版本的nano的性能拉跨，**

**达到一个令人满意的FPS，检测简单手势，**

**然后再通过nano控制beep等外设，来达到beep的变频输出**

**使用7种手势代表7种音阶**

**do、re、mi、fa、sol、la、si**

**且为了使得音调更加丰富，我使用了第八种手势，当第八种手势出现在画面中的时候，音阶就会上抬一个八度,于是我们获得了十四个音阶**

**do、re、mi、fa、sol、la、si、ddo、dre、dmi、dfa、dsol、dla、dsi**

**从左向右从上往下依次是12 .....最后一个是音阶抬高记号**

![](https://ai-studio-static-online.cdn.bcebos.com/d50c1e8b7da04d5093b6ba37786e4fd83bf4d9cd2a7b4f348460b3188d34e15a)






# 1.数据集制作

In [ ]:
#在本地上执行
pip install labelimg

安装labelimg

打开labelimg界面如下

![](https://ai-studio-static-online.cdn.bcebos.com/41858aa1fccf42fb86c5e97c201724621ffe28f8c8bb45aeaf28787b78f29f93)

标注完成后voc数据集结构如下

  │├── Annotations**
  
  │├── one0.xml
  
  │├── one1.xml
  
  │├   ...
  
  │├── JPEGImages
  
  │├── one0.jpg
  
  │├── one1.jpg
  
  │├   ...
  
  │├── ImageSets	
  
  │├──create_txt.py**
  
  





# 2.paddledet环境的aistudio部署

打开aistudio

`pip list`

我们的aistudio环境已经配置好了paddle环境，所以不需要我们自己去设置

通过pip list

我们可以看到环境为paddle2.2.0gpu版本

![](https://ai-studio-static-online.cdn.bcebos.com/c2018c1afc0e42fe82c9b2c6a130ad735628c9303a3a460896080214fc7db4b8)


话不多说,首先在终端执行 

`!git clone https://gitee.com/paddlepaddle/PaddleDetection.git`

将paddledetection在gitee的代码拷贝下来

当看见命令行出现下面这串代码时，即表示我们已经完成了代码的拷贝

![](https://ai-studio-static-online.cdn.bcebos.com/9f8d2cc749474d1aa62c3103bdbc70dd977616000e814b7996f554706c62add4)

命令行输入ls进入文件夹

![](https://ai-studio-static-online.cdn.bcebos.com/49cef3e13f654643ae8d8f45e87bfd35eb66b866fc364f1ca304a9cfc7c52b2f)


我们就可以看得到paddledetection的文件夹

`#进入文件夹`

`%cd PaddleDetection/`

`#安装其他依赖`

`!pip install -r requirements.txt`

`#编译安装paddledet`

`%cd PaddleDetection`

`!python setup.py install`

当命令行最后一行执行出现

`Finished processing dependencies for paddledet==2.3.0` 时

至此,环境部署完成

# 3.VOC数据集的上传与训练configs的修改

点击jupyter notebook左上方的

![](https://ai-studio-static-online.cdn.bcebos.com/57782f08dc76488e8ea4ec6ee318315dd87a3b0cc6a642ef902b3aa2be5a3e1e)


此时，数据集已经上载，但若要使用需要将数据集放到PaddleDetection中dataset文件夹下一级的voc文件夹中

退回到home

然后我们在命令行输入

`cp Music.zip /PaddleDetection/dataset/voc`

![](https://ai-studio-static-online.cdn.bcebos.com/115871912dcd4f79815f3b2d8a1358912388b20516f3456096aac49008677f50)

成功地将压缩文件粘贴至目标文件夹下

下面解压

**建议压缩数据集时候压缩成zip格式，然后上传zip，aistudio中不自带unrar命令**

这里不在赘述

使用unzip解压Music.zip

![](https://ai-studio-static-online.cdn.bcebos.com/5ea68787e59d4a7d934b1ca19278804325442a3cb34a4a6eab114c268b8994e6)

完成解压

下面我们使用paddledetection对ppyolo进行训练

进入PaddleDetection文件夹下的config/ppyolo文件夹,选择我们想要的模型

![](https://ai-studio-static-online.cdn.bcebos.com/32db15a05e9c434d83a68a84cab8eaee3cf2b2317e25477f8a87a59ad7a141b5)

这里我选择ppyolo_r50vd_dcn，使用vim打开对应的yml文件

输入命令

`vim ppyolo_r50vd_dcn_voc.yml`


![](https://ai-studio-static-online.cdn.bcebos.com/2e5ba1fb84554d6286b6649f92b7f6242aa9eea76aa34679bc4f13f4ae850229)

可以看到__BASE__下的是该模型所对应的configs

![](https://ai-studio-static-online.cdn.bcebos.com/21c61b312aab42fc9fde3c6c77294dbac8bf5bf69c794c41a5ce5aa759864ba9)

这些config对应的是训练时的各种设置，至于具体内容对应什么，可以翻阅paddle的[官方文档](https://gitee.com/paddlepaddle/PaddleDetection/blob/release/2.3/docs/tutorials/GETTING_STARTED_cn.md)

我们需要对voc.yml的文件内容做一点更改

进入对应的文件目录，使用vim打开voc.yml

![](https://ai-studio-static-online.cdn.bcebos.com/842e080988d441ba9b6f44a2f93dd63b6c73a17e11be4e5084be0791e25a8d59)

更改为如下代码

```
metric: VOC
map_type: 11point
num_classes: 8

TrainDataset:
  !VOCDataSet
    dataset_dir: ../dataset/voc/Music
    anno_path: ImageSets/train.txt
    label_list: ImageSets/label_list.txt
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult']

EvalDataset:
  !VOCDataSet
    dataset_dir: ../dataset/voc/Music
    anno_path: ImageSets/val.txt
    label_list: ImageSets/label_list.txt
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult']

TestDataset:
  !ImageFolder
    anno_path: ../dataset/voc/Music/ImageSets/label_list.txt
```


因为我制作的数据集地址是在原定基础上多出了一个Music文件夹，然后num_classes需要更改为8个，因为八种手势对应着8种音符

然后我们还需要执行以下Music文件夹下的create_txt.py文件，因为数据集是从我电脑上移植过来的，所以annotation中所对应的图片地址都是我的电脑上的，所以如果需要正确训练，需要将地址更改为aistudio上的对应地址


* create_txt.py长这样

```
import random
import os
#生成train.txt和val.txt
random.seed(2021)
xml_dir  = r'./Annotations'#标签文件地址
img_dir = r'./JPEGImages'#图像文件地址
path_list = list()
for img in os.listdir(img_dir):
    img_path = os.path.join(img_dir,img)
    xml_path = os.path.join(xml_dir,img.replace('jpg', 'xml'))
    path_list.append((img_path, xml_path))
random.shuffle(path_list)
ratio = 0.9
train_f = open(r'./ImageSets/train.txt','w') #生成训练文件
val_f = open(r'./ImageSets/val.txt' ,'w')#生成验证文件

for i ,content in enumerate(path_list):
    img, xml = content
    text = img + ' ' + xml + '\n'
    if i < len(path_list) * ratio:
        train_f.write(text)
    else:
        val_f.write(text)
train_f.close()
val_f.close()

#生成标签文档
label = ['one','two','three','four','five','six','seven','eight']#设置你想检测的类别
with open(r'./ImageSets/label_list.txt', 'w') as f:
    for text in label:
        f.write(text+'\n')
```


# 4.开始训练！
进入tools文件夹输入命令

`python train -c ../configs/ppyolo/ppyolo_r50vd_dcn_voc.yml`


开始激动人心的训练！

![](https://ai-studio-static-online.cdn.bcebos.com/ffdb78358e3c4bda8b7ecc110772084425589d75fcf04df4824cb47c02341acb)

训练完成，使用infer.py进行预测

在tool文件夹下输入命令

`python infer.py -c ../configs/ppyolo/ppyolo_r50vd_dcn_voc.yml --infer_img=one.jpg`

* 这里我把测试图片放在了tool文件夹下面了

出现以下内容

![](https://ai-studio-static-online.cdn.bcebos.com/b0a30e97016a4cfa8bf0334c81e9dee1359e68836b754374abcf7a536bb693f6)

完成预测

预测结果如下：（注：后面更换了数据集，所以跟一开始这张图片不一样）

![](https://ai-studio-static-online.cdn.bcebos.com/c62b3c1de0154ab2ba1c8915f30dc54c31ff7a299a4b4222afc352479186ef23)



# 5.进行模型评估

在tools文件夹下，使用命令

`python eval.py -c ../configs/ppyolo/ppyolo_r50vd_dcn_voc.yml`

我们可以看见eval的结果，mAP达到了%94.82

目前采样了24张图像，预计视频检测帧能达到25FPS(aistudio上)

![](https://ai-studio-static-online.cdn.bcebos.com/63d2e0cbeec04cfabecc29678a9b9185d379858c1fd746908a825b22fd7091af)

# 6.JetsonNano的部署

我这里有一块上好的JetsonNano 2GB版，在同一局域网下使用ssh连接

使用

`nvcc -V`

来查看cuda版本

![](https://ai-studio-static-online.cdn.bcebos.com/3719362c465246a2aeb5c55ccef2aa9ec2416d7a555548c2a5269ed7d12522eb)

这里可以看到我的cuda是10.2的

* 首先我们来部署paddledet

如同aistudio上部署一样，我们依然需要先git clone以下paddledetection的库这里不再赘述

`git clone https://gitee.com/paddlepaddle/PaddleDetection.git`

依然是

`#进入文件夹`

`cd PaddleDetection/`

`#安装其他依赖`

`pip3 install -r requirements.txt`

`#编译安装paddledet`

`cd PaddleDetection`

`python3 setup.py install`

当命令行最后一行执行出现

`Finished processing dependencies for paddledet==2.3.0` 

完成安装

通过后可以通过

`python ppdet/modeling/tests/test_architectures.py`

检测一下是否成功

出现下图时即可判定成功

按照官方给的[教程](https://gitee.com/paddlepaddle/PaddleDetection/blob/release/2.3/docs/tutorials/INSTALL_cn.md)来即可，或者好像可以直接pip安装包，连编译都不要

![](https://ai-studio-static-online.cdn.bcebos.com/e57463f46ea54a399599c6d0f48fa0c9d3c0d8ae6da7457a992e67c63482416c)

* 2G没使用Tensorrt的算力就是弱，跑了我46秒多

这里可以看见报了个Warning，不过不用担心，只要不是Error就可以当作看不见🤭

现在可以把我们训练好的模型放入进行预测，不过得先更改一下configs配置的相关内容

找到configs下datasets中的voc.yml

`vim voc.yml`

![](https://ai-studio-static-online.cdn.bcebos.com/ffd28d2043b7415ba6cb6e795089c6ff85020c5c76d44b51897b95e78e4c18bd)

更改为

![](https://ai-studio-static-online.cdn.bcebos.com/c04a9e2469d843ddb472a4dd7c89328fdb47d1951514465492f452a8e5488b64)

就是地址改一改，classnum改一改

别忘记把labellist放上去，这个时候不用放数据集了

训练好的权重放在一个自己喜欢的地方即可，等会命令行调用的时候改一下地址即可

在tool文件夹下执行

`python3 infer.py -c ../configs/ppyolo/ppyolo_r50vd_dcn_voc.yml --infer_img=one1.jpg -o weights=model_final.pdparams`

注意，jetsonnano中直接使用python是调用python2，而使用python3的时候才是调用python3的

* 不过，性能实在太差，运行了6分种左右被系统Killed了

部署还是没有问题的，这也就凸显出tensorrt的必要了

* 下面进行paddleinference的部署（可以调用tensorrt进行加速）


在aistudio中进行模型的导出


`python export_model.py -c ../configs/yolov3/yolov3_darknet53_270e_voc.yml  -o weights=model_final.pdparams`

将模型导出到

inference_model/yolov3_darknet53_270e_voc文件夹下

分别为
infer_cfg.yml, model.pdiparams, model.pdiparams.info, model.pdmodel

通过使用这些导出的模型文件可以使用Jetson nano带的Tensorrt加速


下面进行paddle_inference的部署

使用 `git clone https://gitee.com/myxxr/paddle-inference.git`

测试环境

进入jetson_inference文件夹的yolov3例程

![](https://ai-studio-static-online.cdn.bcebos.com/e4bc5849847e4697b8a028555aa24d583899f9b7ad094e8b9181d29e088a974f)

运行bash run.sh将例程跑通就可以开始，或者可以跑resnet50的例程

注意，如果报缺少包的错误则需要将run.sh文件中执行的python 改为python3


单张图像预测demo：

在yolov3目录下创建 predict.py，复制以下代码

下面将给出视频流和加上位机的demo，因为是根据例程改的，改的有些乱所以直接都给出

```
import numpy as np
import argparse
import cv2
from PIL import Image

from paddle.inference import Config
from paddle.inference import create_predictor

from utils import preprocess, draw_bbox


def init_predictor(args):
    if args.model_dir != "":
        config = Config(args.model_dir)
    else:
        config = Config(args.model_file, args.params_file)

    config.enable_memory_optim()
    if args.use_gpu:
        config.enable_use_gpu(500, 0)
    else:
        # If not specific mkldnn, you can set the blas thread.
        # The thread num should not be greater than the number of cores in the CPU.
        config.set_cpu_math_library_num_threads(4)
        config.enable_mkldnn()

    predictor = create_predictor(config)
    return predictor


def run(predictor, img):
    # copy img data to input tensor
    input_names = predictor.get_input_names()
    for i, name in enumerate(input_names):
        input_tensor = predictor.get_input_handle(name)
        input_tensor.reshape(img[i].shape)
        input_tensor.copy_from_cpu(img[i].copy())

    # do the inference
    predictor.run()

    results = []
    # get out data from output tensor
    output_names = predictor.get_output_names()
    for i, name in enumerate(output_names):
        output_tensor = predictor.get_output_handle(name)
        output_data = output_tensor.copy_to_cpu()
        results.append(output_data)
    return results


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_file",
        type=str,
        default="yolov3_darknet53_270e_voc/model.pdmodel",
        help="Model filename, Specify this when your model is a combined model."
    )
    parser.add_argument(
        "--params_file",
        type=str,
        default="yolov3_darknet53_270e_voc/model.pdiparams",
        help=
        "Parameter filename, Specify this when your model is a combined model."
    )
    parser.add_argument(
        "--model_dir",
        type=str,
        default="",
        help=
        "Model dir, If you load a non-combined model, specify the directory of the model."
    )
    parser.add_argument("--use_gpu",
                        type=int,
                        default=0,
                        help="Whether use gpu.")
    return parser.parse_args()


if __name__ == '__main__':
    args = parse_args()
    img_name = 'one1.jpg'
    save_img_name = 'res.jpg'
    im_size = 608
    pred = init_predictor(args)
    img = cv2.imread(img_name)
    data = preprocess(img, im_size)
    scale_factor = np.array([im_size * 1. / img.shape[0], im_size * 1. / img.shape[1]]).reshape((1, 2)).astype(np.float32)
    im_shape = np.array([im_size, im_size]).reshape((1, 2)).astype(np.float32)
    result = run(pred, [im_shape, data, scale_factor])
    img = Image.open(img_name).convert('RGB')
    draw_bbox(img, result[0], save_name=save_img_name)
```

如果训练的模型也是yolov3_darknet53_270e_voc，则直接在相同目录下执行

`python3 predict.py`

如果不是则在命令行中加入相应的文件地址

示例

`python3 predict.py --model_file 相对路径.pdmodel --params_file 相对路径.pdiparams --use_gpu 0`

![](https://ai-studio-static-online.cdn.bcebos.com/f3d2f7426bf443a2bccea208f8bee51407d96ef5b2ae46898d478e52d294f825)

![](https://ai-studio-static-online.cdn.bcebos.com/7a80784efc0e4f20b1b5f248482b63d96bae1aadecb04db69665258c937ee7c6)

运行成功应为上图结果



视频流检测demo：
```
import numpy as np
import argparse
import cv2
from PIL import Image
import time

from paddle.inference import Config
from paddle.inference import create_predictor
from paddle.inference import PrecisionType

from utils import preprocess, draw_bbox,return_bbox


def init_predictor(args):
    if args.model_dir != "":
        config = Config(args.model_dir)
    else:
        config = Config(args.model_file, args.params_file)

    config.enable_memory_optim()
    if args.use_gpu:
        config.switch_ir_optim()
        config.enable_use_gpu(500, 0)
    else:
        # If not specific mkldnn, you can set the blas thread.
        # The thread num should not be greater than the number of cores in the CPU.
        config.set_cpu_math_library_num_threads(4)
        config.enable_mkldnn()
    config.enable_tensorrt_engine(workspace_size=1 << 30, precision_mode=PrecisionType.Half,max_batch_size=1, min_subgraph_size=5, use_static=False, use_calib_mode=False)
    predictor = create_predictor(config)
    return predictor


def run(predictor, img):
    # copy img data to input tensor
    input_names = predictor.get_input_names()
    for i, name in enumerate(input_names):
        input_tensor = predictor.get_input_handle(name)
        input_tensor.reshape(img[i].shape)
        input_tensor.copy_from_cpu(img[i].copy())

    # do the inference
    predictor.run()

    results = []
    # get out data from output tensor
    output_names = predictor.get_output_names()
    for i, name in enumerate(output_names):
        output_tensor = predictor.get_output_handle(name)
        output_data = output_tensor.copy_to_cpu()
        results.append(output_data)
    return results


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_file",
        type=str,
        default="yolov3_mobilenet_v1_ssld_270e_voc/model.pdmodel",
        help="Model filename, Specify this when your model is a combined model."
    )
    parser.add_argument(
        "--params_file",
        type=str,
        default="yolov3_mobilenet_v1_ssld_270e_voc/model.pdiparams",
        help=
        "Parameter filename, Specify this when your model is a combined model."
    )
    parser.add_argument(
        "--model_dir",
        type=str,
        default="",
        help=
        "Model dir, If you load a non-combined model, specify the directory of the model."
    )
    parser.add_argument("--use_gpu",
                        type=int,
                        default=1,
                        help="Whether use gpu.")
    return parser.parse_args()


if __name__ == '__main__':
    args = parse_args()


    im_size =320
    pred = init_predictor(args)

    capture = cv2.VideoCapture(0)
    fps = 0.0
    while(True):
        t1 = time.time()
        ref, img = capture.read()
        if not ref:
            break
        img= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        data = preprocess(img, im_size)
        scale_factor = np.array([im_size * 1. / img.shape[0], im_size * 1. / img.shape[1]]).reshape((1, 2)).astype(np.float32)
        im_shape = np.array([im_size, im_size]).reshape((1, 2)).astype(np.float32)
        result = run(pred, [im_shape, data, scale_factor])
        fps  = ( fps + (1./(time.time()-t1)) ) / 2
        print("fps= %.2f"%(fps))
        return_bbox(result[0]) 
        #print(len(result[0]))
        c = cv2.waitKey(1) & 0xff 
        if c==27:
            capture.release()
            break
	

```

上位机加视频流检测demo：
```
import numpy as np
import argparse
import cv2
from PIL import Image
import time

from paddle.inference import Config
from paddle.inference import create_predictor
from paddle.inference import PrecisionType

from utils import preprocess, draw_bbox
import serial as ser
import struct,time

se = ser.Serial(    
    port="/dev/ttyACM0",
    baudrate=115200,
    bytesize=ser.EIGHTBITS,
    parity=ser.PARITY_NONE,
    stopbits=ser.STOPBITS_ONE
    
)

time.sleep(2)

flag = 0

def init_predictor(args):
    if args.model_dir != "":
        config = Config(args.model_dir)
    else:
        config = Config(args.model_file, args.params_file)

    config.enable_memory_optim()
    if args.use_gpu:
        config.switch_ir_optim()
        config.enable_use_gpu(500, 0)
    else:
        # If not specific mkldnn, you can set the blas thread.
        # The thread num should not be greater than the number of cores in the CPU.
        config.set_cpu_math_library_num_threads(4)
        config.enable_mkldnn()
    #config.enable_tensorrt_engine(workspace_size=1 << 30, precision_mode=PrecisionType.Half,max_batch_size=1, min_subgraph_size=5, use_static=False, use_calib_mode=False)
    predictor = create_predictor(config)
    return predictor

def run(predictor, img):
    # copy img data to input tensor
    input_names = predictor.get_input_names()
    for i, name in enumerate(input_names):
        input_tensor = predictor.get_input_handle(name)
        input_tensor.reshape(img[i].shape)
        input_tensor.copy_from_cpu(img[i].copy())

    # do the inference
    predictor.run()

    results = []
    # get out data from output tensor
    output_names = predictor.get_output_names()
    for i, name in enumerate(output_names):
        output_tensor = predictor.get_output_handle(name)
        output_data = output_tensor.copy_to_cpu()
        results.append(output_data)
    return results


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_file",
        type=str,
        default="ppyolo_mbv3_small_coco/model.pdmodel",
        help="Model filename, Specify this when your model is a combined model."
    )
    parser.add_argument(
        "--params_file",
        type=str,
        default="ppyolo_mbv3_small_coco/model.pdiparams",
        help=
        "Parameter filename, Specify this when your model is a combined model."
    )
    parser.add_argument(
        "--model_dir",
        type=str,
        default="",
        help=
        "Model dir, If you load a non-combined model, specify the directory of the model."
    )
    parser.add_argument("--use_gpu",
                        type=int,
                        default=0,
                        help="Whether use gpu.")
    return parser.parse_args()

def return_bbox(result, threshold=0.5):
    """draw bbox"""
    all_bbox = []
    for res in result:
        cat_id, score, bbox = res[0], res[1], res[2:]
        if score < threshold:
            continue
            return all_bbox
        xmin, ymin, xmax, ymax = bbox
        all_bbox.append({"id":cat_id,"score":score,"location":bbox  })
        print('category id is {}, bbox is {}'.format(cat_id, bbox))
        return all_bbox
        #if len(all_bbox):
        #        print(int(all_bbox[-1]['id']))
        #        if flag!=int(all_bbox[-1]['id']):            
        #                se.write((str(int(all_bbox[-1]['id'])+1)+"\n").encode())
        #        flag = int(all_bbox[-1]['id'])
    
    

if __name__ == '__main__':
    args = parse_args()
    im_size =320
    pred = init_predictor(args)
    all_bbox = []	
    capture = cv2.VideoCapture(0)
    fps = 0.0
    while(True):
        t1 = time.time()
        ref, img = capture.read()
        if not ref:
            break
        img= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        data = preprocess(img, im_size)
        scale_factor = np.array([im_size * 1. / img.shape[0], im_size * 1. / img.shape[1]]).reshape((1, 2)).astype(np.float32)
        im_shape = np.array([im_size, im_size]).reshape((1, 2)).astype(np.float32)
        result = run(pred, [im_shape, data, scale_factor])
        fps  = ( fps + (1./(time.time()-t1)) ) / 2
        print("fps= %.2f"%(fps))
        all_box = return_bbox(result[0]) 
        if len(all_bbox)>0:
                print(int(all_bbox[-1]['id']))
                if flag!=int(all_bbox[-1]['id']):            
                        se.write((str(int(all_bbox[-1]['id'])+1)+"\n").encode())
                flag = int(all_bbox[-1]['id'])

        
        #print(len(result[0]))
        c = cv2.waitKey(1) & 0xff 
        if c==27:
            capture.release()
            break
```



最后的文件夹结构是

||

||---weight

||

||---util.py

||

||---video.py

||

其中weight是经paddledetection导出后的权重模型
video.py是你使用的视频流检测demo还是单张图检测demo还是带上位机的部分


util.py代码
```
import cv2
import numpy as np
from PIL import Image, ImageDraw


def resize(img, target_size):
    """resize to target size"""
    if not isinstance(img, np.ndarray):
        raise TypeError('image type is not numpy.')
    im_shape = img.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])
    im_scale_x = float(target_size) / float(im_shape[1])
    im_scale_y = float(target_size) / float(im_shape[0])
    img = cv2.resize(img, None, None, fx=im_scale_x, fy=im_scale_y)
    return img


def normalize(img, mean, std):
    img = img / 255.0
    mean = np.array(mean)[np.newaxis, np.newaxis, :]
    std = np.array(std)[np.newaxis, np.newaxis, :]
    img -= mean
    img /= std
    return img


def preprocess(img, img_size):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    img = resize(img, img_size)
    img = img[:, :, ::-1].astype('float32')  # bgr -> rgb
    img = normalize(img, mean, std)
    img = img.transpose((2, 0, 1))  # hwc -> chw
    return img[np.newaxis, :]


def draw_bbox(img, result, threshold=0.5, save_name='res.jpg'):
    """draw bbox"""
    draw = ImageDraw.Draw(img)
    for res in result:
        cat_id, score, bbox = res[0], res[1], res[2:]
        if score < threshold:
            continue
        xmin, ymin, xmax, ymax = bbox
        draw.line([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin),
                   (xmin, ymin)],
                  width=2,
                  fill=(255, 0, 0))
        print('category id is {}, bbox is {}'.format(cat_id, bbox))
    img.save(save_name, quality=95)

def return_bbox(result, threshold=0.5):
    """draw bbox"""
    all_bbox = []
    for res in result:
        cat_id, score, bbox = res[0], res[1], res[2:]
        if score < threshold:
            continue
        xmin, ymin, xmax, ymax = bbox
        all_bbox.append({"id":cat_id,"score":score,"location":bbox  })
        print('category id is {}, bbox is {}'.format(cat_id, bbox))
    return all_bbox
```

# 7.Arduino部分代码

Arduino使用到了PWM库

PWM库给出网盘链接

[链接：https://pan.baidu.com/s/1pEHHU1nfRWSwO00rkM0WOg 
提取码：haha](http://)

```
#include <PWM.h>

#define dDo 175 
#define dRe 196  
#define dMi 221  
#define dFa 234  
#define dSol 262  
#define dLa 294
#define dSi 330

#define Do 350
#define Re 393  
#define Mi 441 
#define Fa 495  
#define Sol 556
#define La 624
#define Si 661 

int pin = 9;                // led所连接到的引脚
int brightness = 100;         // led的亮度，也就是占空比，范围是0-255
//0-6,7-14,14-20
int32_t fre[21]={dDo,dRe,dMi,dFa,dSol,dLa,dSi,Do,Re,Mi,Fa,Sol,La,Si};

int message = 0;//用于接收信息进行对比

//void cf(int32_t mark,int32_t last,int32_t h_l=1)
void cf(int32_t mark)//mark 为标号，last 为持续时长
{
    mark-=1;  
    bool success = SetPinFrequencySafe(pin, fre[mark]);
    pwmWrite(pin, brightness);
}

void setup()
{
  // 初始化除了0号计时器以外的其他计时器
  InitTimersSafe(); 

  // 设置指定引脚的频率
  bool success = SetPinFrequencySafe(pin, dDo);

  delay(300);
  Serial.begin(9600);//初始化串口函数
  

}

void loop()
{
  //Serial.println("6");
//  Serial.println(Serial.parseInt());

  if (Serial.available() > 0)//串口接收到数据
  {
      int incomedate = Serial.parseInt();//获取串口接收到的数据
      Serial.println(incomedate);
//      pwmWrite(pin, brightness);
//      delay(300);
      if (incomedate>0){
        cf(incomedate);

        delay(300);
        Serial.println(incomedate);
        pwmWrite(pin, 0);

      }

  }
}
```

B站视频链接https://www.bilibili.com/video/BV1134y1q7FY?spm_id_from=333.999.0.0

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 